# Geopandas Tutorial
## [documentation](http://geopandas.org/index.html)
## introduction:

Python處理空間資料有許多套件，這些套件都有不同的使用場景，例如shapely用在向量資料的處理、fiona則是資料格式轉換與I/O，pyproj做坐標投影的轉換等等。

### 這麼多套件要搞，用python處理空間資料分析有夠麻煩...

`Geopandas`讓分析空間資料就像用Pandas一樣，它主要依賴了以下Python空間資料處理的套件：
- shapely
- fiona
- descartes
- matplotlib
- pyproj
- numpy
- more...

In [ ]:
# 先載入一些本堂課會用的套件
import os
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline  

## 讀寫GIS資料

可以利用GeoPandas讀取shp資料，以進行我們想要的各種處理或分析，除了shp外，Geopandas也支援geojson等相關空間資料檔案格式，以[縣市界線圖 | 政府資料開放平臺](https://data.gov.tw/dataset/7442)這個資料集為例，我們將其下載到本機解壓縮，使用GeoPandas讀取為GeoDataFrame

In [ ]:
TW_gdf = gpd.read_file('data/TW/COUNTY_MOI_1070516.shp', encoding='utf-8') # shapefile, GeoJSON...
TW_gdf.head()

### 支援pandas的各種操作方式

In [ ]:
TW_gdf['COUNTYNAME'].unique()

In [ ]:
TW_gdf[TW_gdf['COUNTYNAME'] == '宜蘭縣']

In [ ]:
TW_gdf.sort_values(by='COUNTYID')

In [ ]:
TW_gdf.plot()

In [ ]:
TW_gdf.plot(column='COUNTYENG', figsize=(12, 12), legend=True)

## 從Pandas到Geopandas

對於DataFrame或者GeoDataFrame的操作，可以參考[Data wrangling with Pandas](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)的圖表。

Geopandas可以讀取各GIS格式進行空間運算，然而實際上我們可能拿到的資料不見得是GIS的格式,雖然不是GIS格式，但它有可能是有坐標資訊的資料
這樣的資料可能會是一個 `csv`或`Excel`，資料中含有經緯度或x,y，對於這樣的資料，我們可以把它先利用`pandas`讀取檔案再轉為`Geopandas`的`GeoDataFrame`以方便空間的運算。

### 用Pandas讀取csv或Excel

請先下載[全台空氣品質指標(AQI)](https://opendata.epa.gov.tw/Data/Contents/AQI/)的csv檔，拿到資料的第一步先使用Pandas讀取這個csv為`DataFrame`

In [ ]:
df = pd.read_csv('data/AQI_20190304161106.csv',encoding='utf-8')
df.head()

### DataFrame轉為Geodataframe

要產生`GeoDataFrame`，需要產生`geometry`，這時候需要用`shapely`套件中的`geometry`，我們使用之中的`Point`類型，並把`Longitude`, `Latitude`包進去

In [ ]:
from shapely.geometry import Point

geom = [Point(point) for point in zip(df.Longitude, df.Latitude)]

有了`geom`這個 `Point`類型的變數，我們就可以用來產生`GeoDataFrame`，其中，需要指定`crs`，也就是坐標系統由於TWD97X, TWD97Y的坐標是`WGS84`，為`epsg:4326`

In [ ]:
crs = 'epsg:4326'
AQI_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geom)
AQI_gdf.head()

In [ ]:
AQI_gdf.plot('AQI', figsize=(10, 10), legend=True)

In [ ]:
directory = './output'
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
AQI_gdf.to_file(driver='ESRI Shapefile', filename=directory+'/AQI.shp', encoding='utf-8')

## GIS資料空間運算

### 很好我們會讀取資料跟基本操作了，那還有什麼跟 `pandas`不一樣的？
- 座標轉換
- 幾何操作
- 空間運算子

### 座標轉換
Geopandas的座標轉換依賴pyrpoj套件，pyrpoj是坐標轉換非常方便的工具，因此，在Geopandas坐標轉換變得很簡單，使用剛剛的`AOI_gpf`做示範，因為它是`epsg4326(WGS84)`，我們會試著轉成`epsg3826(TWD97)`

In [ ]:
AQI_gdf.to_crs(epsg=3826, inplace=True)
AQI_gdf.head()

#### 可以看到坐標系統已經成功轉換為TWD97(4326-->3826)

#### 例題：
請各位同學試著把上面縣市界線圖所匯入的`TW_gdf`坐標係由`WSG84`轉為`TWD97`

### 幾何操作
Geopandas的幾何操作主要是來自shaply套件，以下來試看看在GIS軟體常用的功能：
- buffer
- area
- envelope
- convex_hull

#### Buffer
在GIS中常用來分析點線面的影響範圍

In [ ]:
base = AQI_gdf.plot(color='red', figsize=(12, 12))
AQI_gdf.buffer(10000).plot(ax=base, color='blue', alpha=0.3)

#### Area

area是GeoDataFrame中計算每一筆幾何資料面積的方法

In [ ]:
buffer = AQI_gdf.head(1).buffer(100)
area = buffer.area
print(area[0])

#### Envelope
GeoDataFrame中，每一筆資料包覆的長方形範圍，是一個四角坐標

In [ ]:
TW_env = TW_gdf[['COUNTYENG', 'geometry']]
TW_env['geometry'] = TW_env['geometry'].envelope
TW_env.plot('COUNTYENG', figsize=(12, 12), legend=True)

#### Convex_hull
convex hull與envelope類似但不一樣，它是包住每一個資料的凸殼多邊形

In [ ]:
TW_con = TW_gdf[['COUNTYENG', 'geometry']]
TW_con['geometry'] = TW_con['geometry'].convex_hull
TW_con.plot('COUNTYENG', figsize=(12, 12), legend=True)

### 幾何轉換
Geodataframe可以進行幾何的投影轉換，包含了平移、尺度縮放、旋轉等，這邊舉尺度縮放、旋轉與平移的例子

分別在x方向10倍及y方向5倍的投影，並與原本的buffer圖比較。

In [ ]:
envelope = AQI_gdf.head(1).buffer(500).envelope
scaled = envelope.scale(10, 5).plot(color='green', alpha=0.5)
envelope.plot(ax=scaled, color='blue')

In [ ]:
envelope = AQI_gdf.head(1).buffer(500).envelope
rotated = envelope.rotate(45).plot(color='green', alpha=0.5)
envelope.plot(ax=rotated, color='blue', alpha=0.5)

In [ ]:
buffer = AQI_gdf.head(1).buffer(500)
translated = buffer.translate(xoff=800.0, yoff=0.0).plot(color='green', alpha=0.5)
buffer.plot(ax=translated, color='blue', alpha=0.5)

其餘的一些操作在Shapely的官方文件有滿多說明的，特別是對幾何資料的一些檢查，建議有需要時瀏覽一遍Shapely的參考文件[shapely documentation](https://shapely.readthedocs.io/en/stable/)

### 空間運算子
空間運算子是屬於資料與資料之間的運算與分析常見的GIS運算包含了幾項運算子，如:
![From GeoPandas documentation](http://geopandas.org/_images/overlay_operations.png)
*From GeoPandas documentation*

In [ ]:
p1 = gpd.GeoDataFrame(geometry=[Point([(0, 0)])])
p2 = gpd.GeoDataFrame(geometry=[Point([(600, 0)])])

p1['geometry'] = p1.buffer(500)
p2['geometry'] = p2.buffer(500)

base = p1.plot(color='blue')
p2.plot(ax=base, color='green')

### Intersection
Intersection可以算出兩個圖形的交集，我們分別使用不同的顏色給p1與p2，並把交集的部分給定黃色

In [ ]:
intersection = gpd.overlay(p1, p2, how='intersection')
base = p1.plot(color='blue')
p2.plot(ax=base, color='green')
intersection.plot(ax=base,color='yellow')

In [ ]:
intersection.area

### Union
union是計算聯集黃色的部分，就是p1, p2聯集的部分

In [ ]:
union = gpd.overlay(p1, p2, how='union')
base = p1.plot(color='blue')
p2.plot(ax=base, color='green')
union.plot(ax=base,color='yellow')

In [ ]:
union.area

### Difference
difference會算出兩個幾何的差異，以黃色表示

In [ ]:
difference = gpd.overlay(p1, p2, how='difference')
base = p1.plot(color='blue')
p2.plot(ax=base, color='green')
difference.plot(ax=base, color='yellow')

In [ ]:
difference.area

## Geopandas的資料聚合
`pandas`資料聚合的方式主要為`concat`, `merge`的應用，前述已經介紹過了...`geopandas`也有針對空間資訊的資料聚合方法：
- spatial join
- dissolve

### spatial join
利用水源局的一段`code`來介紹

In [ ]:
ls_pts = gpd.read_file('data/aere_grid_2004082422.geojson')

slope_unit = gpd.read_file(os.path.join("data", 'slope_unit_xn.shp'), crs={'init' :'epsg:3826'})
slope_unit = slope_unit[~(slope_unit['SLOPE_UNIT'].isna())]

In [ ]:
ls_pts.head()

In [ ]:
slope_unit.head()

In [ ]:
slope_unit.plot('SLOPE_UNIT', figsize=(7,6))

In [ ]:
result = gpd.sjoin(ls_pts, slope_unit[['SLOPE_UNIT', 'geometry']], how="right", op='within')
result = result[~(result['x'].isna())]

其中，`how`是選擇要`left join`, `right join`或`inner join`，`op`則是空間關聯的方法，有`contains`, `within`, `intersects`可以選擇，我們選`within`，因為我們想計算點被包含在哪些縣市界中。

`spatial join`完成後，我們就可以做`groupy`計算每個縣市的測站數量

In [ ]:
result.head()

In [ ]:
result_group = result[['SLOPE_UNIT', 'pred', 'depth']].groupby('SLOPE_UNIT', as_index=False)

su_result = result_group.mean()
su_result['p_max'] = result_group.max()['pred']
su_result['counts'] = result_group.size().values
su_result['area(m2)'] = su_result['counts'] * 400
su_result['depth'] = su_result['depth'] * su_result['area(m2)']

su_result.rename(columns={'pred': 'p_mean', 'depth': 'volume(m3)'}, inplace=True)
su_result = su_result.merge(slope_unit[['SLOPE_UNIT', 'geometry']], on=['SLOPE_UNIT'], how='left')
su_result = gpd.GeoDataFrame(su_result)

In [ ]:
su_result.head()

In [ ]:
base = slope_unit.plot(color='white', edgecolor='black', figsize=(12, 12))
su_result.plot('counts', ax=base, legend=True)
plt.xlim((290000, 310000))
plt.ylim((2730000, 2750000))

### dissolve
試著把上面台灣縣市界資料做一次聚合，使用分區上色，會用到`dissolve`，其中第一個參數是要dissolve的欄位，aggfunc則是欄位中被合併的資料呈現的方式，我們這邊的聚合方式是`sum`，首先我們先對`TW_gdf`做一個新的欄位

In [ ]:
北部 = ['基隆市', '新竹市', '臺北市', '新北市', '桃園市', '苗栗縣', '新竹縣',]
中部 = ['彰化縣', '南投縣', '雲林縣', '臺中市',]
南部 = ['屏東縣', '臺南市', '嘉義市', '嘉義縣', '高雄市',]
東部 = ['宜蘭縣', '臺東縣', '花蓮縣',]
離島 = ['連江縣', '澎湖縣', '金門縣']
TW_gdf['分區'] = 0
for z in 北部:
    TW_gdf.loc[TW_gdf.COUNTYNAME == z, '分區'] = 'North'
for z in 中部:
    TW_gdf.loc[TW_gdf.COUNTYNAME == z, '分區'] = 'Central'
for z in 南部:
    TW_gdf.loc[TW_gdf.COUNTYNAME == z, '分區'] = 'South'
for z in 東部:
    TW_gdf.loc[TW_gdf.COUNTYNAME == z, '分區'] = 'East'
for z in 離島:
    TW_gdf.loc[TW_gdf.COUNTYNAME == z, '分區'] = 'Islands'

In [ ]:
TW_gdf.head()

In [ ]:
dis = TW_gdf.dissolve('分區', aggfunc='sum', as_index=False)
dis

In [ ]:
dis.plot('分區' , figsize=(10, 10), legend=True)

## GeoPandas基本繪圖
前面我們有用`Geopandas`裡面包的`matplotlib`做一些基本的繪圖，今天我們以`matplotlib`與`geoplot`，討論一下GIS資料視覺化的一些要素。
### 顏色
資訊圖表的視覺化需考量視覺變數，`顏色`是視覺變數的其中一個項目，在此利用上面的`aggre_v` `geoDataFrame`做一些示範

我們想以`counts`上色，並且控制一些繪圖的參數，第一個可以控制的是`cmap`顏色類型

In [ ]:
dis['area'] = dis.to_crs(epsg=3826)['geometry'].area /10000000

In [ ]:
dis

In [ ]:
dis.plot('area', cmap='YlGn', figsize=(12, 8), legend=True)

給另外一種顏色，[顏色參考](https://matplotlib.org/examples/color/colormaps_reference.html)

In [ ]:
dis.plot('area', cmap='Greys', figsize=(12, 8), legend=True)

另外，還可以調整scheme，包含了`‘equal_interval’`, `‘quantiles’` or `‘fisher_jenks’`，也就是顏色間隔的計算方式 

In [ ]:
dis.plot('area', cmap='Greys', scheme='quantiles', figsize=(12, 8), legend=True)

還有`k`值，給定顏色的數量

In [ ]:
dis.plot('area', cmap='Greys', scheme='quantiles', k=10, figsize=(12, 8), legend=True)

In [ ]:
TW_gdf.plot(column='COUNTYNAME', color='white', edgecolor='black', figsize=(10,10))

### 圖層疊套

In [ ]:
base = TW_gdf.plot(column='COUNTYNAME', color='white', edgecolor='black', figsize=(15,15))
AQI_gdf.to_crs(epsg=4326).plot(ax=base, color='red')

#### 例題：
畫一張分區為北部的圖，並顯示圖上測站點平均PM2.5差異。`hint: 創一個GDF含有AQI資料及對應的*分區*(利用sjoin)`